# Working on Yelp Restaurants Dataset

* To see how the data is genereted by using Yelp API calls, and transformed to csv file. Go to Yelp-api folder, and put your API KEY in to config.py file, select the term "restaurants" then select "search location", and run the search.py file in your environment.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import seaborn as sns
from scipy.stats import linregress
from matplotlib.pyplot import figure
from sklearn import datasets

In [3]:
# File to Load 
yelp_data_to_load = "data/PyRestaurants_data.csv"

In [23]:
# Read Data File and store into Pandas DataFrames
Restaurants_df = pd.read_csv(yelp_data_to_load)
Restaurants_df.head()

,id,name,phone,location.address1,location.address2,location.city,location.state,location.country,price,rating,review_count,url
0,6i7KnRylm4bhPIApDFSVCg,Dave's Hot Chicken,1.323537e+10,970 N Western Ave,NaN,Los Angeles,CA,US,$$,4.5,3132,https://www.yelp.com/biz/daves-hot-chicken-los...
1,qAAoilHU25Qr45FOQlA19g,Kang Ho-dong Baekjeong,1.213385e+10,3465 W 6th St,Ste 20,Los Angeles,CA,US,$$,4.5,4266,https://www.yelp.com/biz/kang-ho-dong-baekjeon...
2,AIhWsp8yNfKZfaX9d9Yajg,Pasta Sisters,1.323871e+10,3343 W Pico Blvd,NaN,Los Angeles,CA,US,$$,4.5,1479,https://www.yelp.com/biz/pasta-sisters-los-ang...
3,vUe6SGmCpWRbVdmj1ujMMQ,Yeon Uh,1.213568e+10,621 S Western Ave,Ste 208 B,Los Angeles,CA,US,NaN,5.0,4,https://www.yelp.com/biz/yeon-uh-los-angeles-2...
4,H9ZlIqEp930ZRniH6VRp8w,Papa Cristo's,1.323737e+10,2771 W Pico Blvd,NaN,Los Angeles,CA,US,$$,4.5,1911,https://www.yelp.com/biz/papa-cristos-los-ange...


## Cleaning Data
* Rename column headings 
* Remove duplicates 
* Remove null values

In [13]:
Restaurants_df.columns

Index(['id', 'name', 'phone', 'location.address1', 'location.address2',
       'location.city', 'location.state', 'location.country', 'price',
       'rating', 'review_count', 'url'],
      dtype='object')

In [12]:
# Using .rename(columns={}) in order to rename columns
renamed_df = Restaurants_df.rename(columns={"id":"Restaurant ID", "name":"Restaurant Name", "phone":"Phone Number","location.address1":"Adress",
                                          "location.address2":"Address Two","location.city":"City","location.state":"State","location.state":"State",
                                           "price":"Price","rating":"Rating","review_count":"Review Count","url":"Url"})
renamed_df.head()



,Restaurant ID,Restaurant Name,Phone Number,Adress,Address Two,City,State,location.country,Price,Rating,Review Count,Url
0,6i7KnRylm4bhPIApDFSVCg,Dave's Hot Chicken,1.323537e+10,970 N Western Ave,NaN,Los Angeles,CA,US,$$,4.5,3132,https://www.yelp.com/biz/daves-hot-chicken-los...
1,qAAoilHU25Qr45FOQlA19g,Kang Ho-dong Baekjeong,1.213385e+10,3465 W 6th St,Ste 20,Los Angeles,CA,US,$$,4.5,4266,https://www.yelp.com/biz/kang-ho-dong-baekjeon...
2,AIhWsp8yNfKZfaX9d9Yajg,Pasta Sisters,1.323871e+10,3343 W Pico Blvd,NaN,Los Angeles,CA,US,$$,4.5,1479,https://www.yelp.com/biz/pasta-sisters-los-ang...
3,vUe6SGmCpWRbVdmj1ujMMQ,Yeon Uh,1.213568e+10,621 S Western Ave,Ste 208 B,Los Angeles,CA,US,NaN,5.0,4,https://www.yelp.com/biz/yeon-uh-los-angeles-2...
4,H9ZlIqEp930ZRniH6VRp8w,Papa Cristo's,1.323737e+10,2771 W Pico Blvd,NaN,Los Angeles,CA,US,$$,4.5,1911,https://www.yelp.com/biz/papa-cristos-los-ange...


## Yelp Restaurants data summary

* Total number of Restaurants

* Total number of Restaurants in each cities

* The average review_count 

* The avarage rating 

* Percentage of Restaurants rating greater than 3 and review count greater than 4142 (top restaurants)

* Percentage of Restaurants less than 2 and review count less than 500 (lower restaurants)
* some plot here 

In [35]:
Restaurant_count = len(renamed_df["Restaurant Name"].unique())
Restaurant_count

4058

## City Summery 

* overview table that summarizes Restaurants in each cities 


## Review count by cities


## Rating in each cities

## Top resturants

## Bottom resturants

# Bar and Pie Charts


In [14]:
# Generate a bar plot showing the total number of resturants in each cities 

In [15]:
#  Generate a pie plot showing the distribution resturants 1 to 5

## Quartiles, Outliers and Boxplots

In [16]:
# Rating of each resturants across the four cities 

# Scatter plot

In [17]:
# Generate a scatter plot of review count versus rating volume for each cities 

# Correlation and Regression

In [18]:
# Correlation between review count and star rating

In [19]:
# Linear regression between review count and star rating